# About: VMの停止

VMの停止用 Notebook。

## Operation Note

*ここに経緯を記述*

# Notebookと環境のBinding

Inventory中のgroup名でBind対象を指示する。ここでは、**VMを収容している物理マシンを指定する。**

In [1]:
target_group = 'test-hypervisor'

Bind対象への疎通状態を確認する。

In [2]:
!ansible -m ping {target_group}

XXX.XXX.XXX.105 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}


対象マシンにlibvirtがインストールされているかを確認する。

In [3]:
!ansible -b -a 'virsh version' {target_group}

XXX.XXX.XXX.105 | SUCCESS | rc=0 >>
Compiled against library: libvirt 0.10.2
Using library: libvirt 0.10.2
Using API: QEMU 0.10.2
Running hypervisor: QEMU 0.12.1



# 停止対象のVM確認

停止したいVMのIPアドレスを定義する。

In [4]:
target_vm = 'XXX.XXX.XXX.66'

対象にpingが通ることを確認する。

In [5]:
!ansible -m ping {target_vm}

XXX.XXX.XXX.66 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}


対象のVM名を検索する。

In [6]:
vmnames = !ansible -b -a 'virsh list' {target_group}
vmnames = vmnames[1:]
vmnames = vmnames[map(lambda l: l[0], filter(lambda l: l[1].startswith('----'), enumerate(vmnames)))[0] + 1:]
vmnames = map(lambda l: l.split()[1], vmnames)
vmnames

['testvm-001']

In [7]:
import re
def get_mac_address(vmname):
    domiflist_stdio = !ansible -b -a "virsh domiflist {vmname}" {target_group}
    mac_pattern = re.compile(r'.*bridge.*\s([0-9a-f\:]+)\s*')
    vmmac = [mac_pattern.match(line).group(1) for line in domiflist_stdio if mac_pattern.match(line)][0]
    return vmmac

def get_ip_address(vmmac):
    leases_stdio = !ansible -b -a "grep {vmmac} /var/lib/dnsmasq/dnsmasq.leases" {target_group}
    ip_pattern = re.compile(r'.*\s([0-9a-f\:]+)\s+([0-9\.]+)\s.*')
    ipaddr = [ip_pattern.match(line).group(2) for line in leases_stdio if ip_pattern.match(line)][0]
    return ipaddr

In [8]:
vmdescs = zip(vmnames, map(lambda mac: get_ip_address(mac), map(lambda n: get_mac_address(n), vmnames)))
vmdescs

[('testvm-001', 'XXX.XXX.XXX.66')]

対象のVM名を取得する。

In [9]:
vmname = filter(lambda e: e[1] == target_vm, vmdescs)[0][0]
vmname

'testvm-001'

VMの状態の確認。runningならば想定通り。

In [10]:
!ansible -b -a "virsh domstate {vmname}" {target_group}

XXX.XXX.XXX.105 | SUCCESS | rc=0 >>
running



# 仮想マシンの停止

仮想マシンを停止する。

In [11]:
!ansible -b -a "virsh destroy {vmname}" {target_group}

XXX.XXX.XXX.105 | SUCCESS | rc=0 >>
Domain testvm-001 destroyed



しばらく待ってから再度 virsh listを実行すると、仮想マシンが停止してリストから消えたことがわかる。

In [12]:
!ansible -b -a "virsh list" {target_group}

XXX.XXX.XXX.105 | SUCCESS | rc=0 >>
 Id    Name                           State
----------------------------------------------------



## dnsmasqの後始末

dnsmasqのリース情報の後始末。VM用IPアドレスが潤沢にある場合は不要。

In [13]:
!ansible -a "cat /var/lib/dnsmasq/dnsmasq.leases" {target_group}

XXX.XXX.XXX.105 | SUCCESS | rc=0 >>
1466235520 XX:XX:XX:XX:XX:XX XXX.XXX.XXX.66 ubuntu *



In [14]:
!ansible -b -m lineinfile -a "dest=/var/lib/dnsmasq/dnsmasq.leases regexp='^.*\s+{ target_vm }\s+.*' state=absent" {target_group}

XXX.XXX.XXX.105 | SUCCESS => {
    "backup": "", 
    "changed": true, 
    "found": 1, 
    "msg": "1 line(s) removed"
}


In [15]:
!ansible -a "cat /var/lib/dnsmasq/dnsmasq.leases" {target_group}

XXX.XXX.XXX.105 | SUCCESS | rc=0 >>




In [16]:
!ansible -b -m service -a "name=dnsmasq state=restarted" {target_group}

XXX.XXX.XXX.105 | SUCCESS => {
    "changed": true, 
    "name": "dnsmasq", 
    "state": "started"
}


# Inventoryの更新

Inventoryから、破棄したマシンのIPアドレスを削除する。変更する前に、現在の内容をコピーしておく。

In [17]:
import tempfile
work_dir = tempfile.mkdtemp()
work_dir

'/tmp/tmp5Wl4z9'

In [18]:
!cp inventory {work_dir}/inventory-old

[Inventory](../edit/inventory) を修正する。

In [19]:
!diff -ur {work_dir}/inventory-old inventory

--- /tmp/tmp5Wl4z9/inventory-old	2016-06-18 05:25:51.809190153 +0900
+++ inventory	2016-06-18 05:25:57.569237254 +0900
@@ -1,5 +1,2 @@
 [test-hypervisor]
 XXX.XXX.XXX.105
-
-[test-vm]
-XXX.XXX.XXX.66


削除したVMに**pingが通じない**かどうかを確認する。

In [20]:
!ansible -m ping {target_vm}

 [WARNING]: provided hosts list is empty, only localhost is available



# 後始末

一時ディレクトリを削除する。

In [21]:
!rm -fr {work_dir}